# Cargar y preparar el dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import numpy as np
import random
from transformers import set_seed

def fix_all_seeds(seed=42):
    # Semilla para librerías estándar
    random.seed(seed)
    np.random.seed(seed)

    # Semilla para PyTorch (CPU y GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Semilla específica de Transformers (afecta a inicialización de pesos y dropout)
    set_seed(seed)

    # Garantizar determinismo en algoritmos de la GPU (puede ralentizar un poco)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print(f"Semilla {seed} fijada globalmente.")

fix_all_seeds(42)

Semilla 42 fijada globalmente.


In [ ]:
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import GroupShuffleSplit

def load_and_split_grouped_dataset(path, test_size=0.1, val_size=0.1, seed=42):
    # 1. Cargar el archivo JSONL a un DataFrame de Pandas
    with open(path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    df = pd.DataFrame(data)

   # 1. División Train vs Test
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    train_idx, test_idx = next(gss.split(df, groups=df['group_id']))
    df_train_total = df.iloc[train_idx]
    df_test = df.iloc[test_idx]

    # 2. División Train vs Validation
    val_relative_size = val_size / (1 - test_size)
    gss_val = GroupShuffleSplit(n_splits=1, test_size=val_relative_size, random_state=seed)
    train_final_idx, val_idx = next(gss_val.split(df_train_total, groups=df_train_total['group_id']))

    # 3. BARAJADO EXPLÍCITO
    # .sample(frac=1) desordena todas las filas. reset_index evita conflictos de índices.
    train_df = df_train_total.iloc[train_final_idx].sample(frac=1, random_state=seed).reset_index(drop=True)
    val_df = df_train_total.iloc[val_idx].sample(frac=1, random_state=seed).reset_index(drop=True)
    test_df = df_test.sample(frac=1, random_state=seed).reset_index(drop=True)

    return DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'validation': Dataset.from_pandas(val_df),
        'test': Dataset.from_pandas(test_df)
    })

path_dataset = '/content/drive/MyDrive/TFG/multimodal_dataset_lite.jsonl'
dataset = load_and_split_grouped_dataset(path_dataset)

In [ ]:
# Comptobacion
train_ids = set(dataset['train']['group_id'])
test_ids = set(dataset['test']['group_id'])

# La intersección debe ser un conjunto vacío
overlap = train_ids.intersection(test_ids)
print(f"Coincidencias de IDs entre train y test: {len(overlap)}")

Coincidencias de IDs entre train y test: 0


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    texts = [f"{t} [SEP] {c}" for t, c in zip(examples['title'], examples['title'])]
    tokenized_inputs = tokenizer(texts, truncation=True, padding='max_length', max_length=512)

    # Casting explícito a int para evitar NaNs y errores de interpretación
    #tokenized_inputs["labels"] = [int(label) for label in examples["is_real"]]
    tokenized_inputs["labels"] = examples["is_real"]
    return tokenized_inputs
'''
def preprocess_function(examples):
    # Opcional: Limpiar artefactos para forzar aprendizaje semántico
    clean_contents = [re.sub(r'\n\n|\n|\s{2,}', ' ', c) for c in examples['title']]

    # Combinamos titular y contenido limpio
    texts = [f"{t} [SEP] {c}" for t, c in zip(examples['title'], clean_contents)]
    tokenized_inputs = tokenizer(texts, truncation=True, padding='max_length', max_length=512)
    tokenized_inputs["labels"] = examples["is_real"]
    return tokenized_inputs
'''
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(f"Dataset features: {tokenized_dataset['train'].features}")

<>:17: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1879/1823440543.py:17: SyntaxWarning: invalid escape sequence '\s'
  clean_contents = [re.sub(r'\n\n|\n|\s{2,}', ' ', c) for c in examples['content']]


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset features: {'group_id': Value('string'), 'is_real': Value('int64'), 'title': Value('string'), 'content': Value('string'), 'image_path': Value('string'), 'model': Value('string'), 'technique': Value('string'), 'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8')), 'labels': Value('int64')}


In [6]:
print(tokenized_dataset['validation']['labels'][:100])

[1, 0, 0, 1, 1, 0]


# Cargar modelo y ejecutar entrenamiento


## Definir funciones de metricas

In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00


In [8]:
import numpy as np
import evaluate

# Cargamos la métrica estándar de clasificación
metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]

    return {"accuracy": acc, "f1": f1}

Para la optimización de hiperparámetros se implementó un Weights & Biases Sweep empleando Optimización Bayesiana. Este enfoque permite modelar la función de pérdida mediante un proceso Gaussiano, optimizando la búsqueda en el espacio de parámetros (Search Space) y convergiendo hacia el mínimo global de forma más eficiente que un muestreo aleatorio (Random Search).

In [11]:
import wandb
import re
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

sweep_config = {
    'method': 'random',
    'metric': {'name': 'eval_f1', 'goal': 'maximize'},
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-6,
            'max': 2e-5
        },
        'weight_decay': {
            'values': [0.01, 0.05, 0.1]
        },
        'num_train_epochs': {
            'values': [3, 5]
        },
        'warmup_ratio': {
            'values': [0.1, 0.15]
        },
        'per_device_train_batch_size': {
            'values': [8, 16]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="tfg-deberta-tuning")

def train_iteration():
    with wandb.init():
        config = wandb.config

        id2label = {0: "SYNTHETIC", 1: "REAL"}
        label2id = {"SYNTHETIC": 0, "REAL": 1}

        model = AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-v3-base",
            num_labels=2,
            id2label=id2label,
            label2id=label2id
        )

        args = TrainingArguments(
            output_dir="./temp_checkpoints",
            report_to="wandb",
            learning_rate=config.learning_rate,
            num_train_epochs=config.num_train_epochs,
            per_device_train_batch_size=config.per_device_train_batch_size,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1"
        )

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        trainer.train()

wandb.agent(sweep_id, function=train_iteration, count=10)

Create sweep with ID: 15zy4s4w
Sweep URL: https://wandb.ai/javierprior04-universidad-de-murcia/tfg-deberta-tuning/sweeps/15zy4s4w


wandb: Agent Starting Run: 5soj9tdv with config:
wandb: 	learning_rate: 1.4621801161577178e-06
wandb: 	num_train_epochs: 5
wandb: 	per_device_train_batch_size: 8
wandb: 	warmup_ratio: 0.15
wandb: 	weight_decay: 0.01
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


Loading weights:   0%|          | 0/198 [00:00<?, ?it/s]

DebertaV2ForSequenceClassification LOAD REPORT from: microsoft/deberta-v3-base
Key                                     | Status     | 
----------------------------------------+------------+-
mask_predictions.classifier.weight      | UNEXPECTED | 
lm_predictions.lm_head.dense.bias       | UNEXPECTED | 
mask_predictions.LayerNorm.bias         | UNEXPECTED | 
lm_predictions.lm_head.bias             | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED | 
lm_predictions.lm_head.dense.weight     | UNEXPECTED | 
mask_predictions.dense.weight           | UNEXPECTED | 
mask_predictions.LayerNorm.weight       | UNEXPECTED | 
mask_predictions.classifier.bias        | UNEXPECTED | 
mask_predictions.dense.bias             | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED | 
classifier.bias                         | MISSING    | 
classifier.weight                       | MISSING    | 
pooler.dense.bias                       | MISSING    | 
pooler.dense.weight      

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.648941,0.666667,0.666667
2,No log,0.618443,0.666667,0.666667
3,No log,0.595309,0.666667,0.666667


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['deberta.embeddings.LayerNorm.weight', 'deberta.embeddings.LayerNorm.bias', 'deberta.encoder.layer.0.attention.output.LayerNorm.weight', 'deberta.encoder.layer.0.attention.output.LayerNorm.bias', 'deberta.encoder.layer.0.output.LayerNorm.weight', 'deberta.encoder.layer.0.output.LayerNorm.bias', 'deberta.encoder.layer.1.attention.output.LayerNorm.weight', 'deberta.encoder.layer.1.attention.output.LayerNorm.bias', 'deberta.encoder.layer.1.output.LayerNorm.weight', 'deberta.encoder.layer.1.output.LayerNorm.bias', 'deberta.encoder.layer.2.attention.output.LayerNorm.weight', 'deberta.encoder.layer.2.attention.output.LayerNorm.bias', 'deberta.encoder.layer.2.output.LayerNorm.weight', 'deberta.encoder.layer.2.output.LayerNorm.bias', 'deberta.encoder.layer.3.attention.output.LayerNorm.weight', 'deberta.encoder.layer.3.attention.output.LayerNorm.bias', 'deberta.encoder.layer.3.output.LayerNorm.weight', 'deberta.encoder.layer.3.output.Laye

eval/accuracy,█▁█
eval/f1,█▁█
eval/loss,█▆▁
eval/runtime,█▅▁
eval/samples_per_second,▁▄█
eval/steps_per_second,▁▄█
train/epoch,▁▅██
train/global_step,▁▅██
eval/accuracy,0.66667
eval/f1,0.5
eval/loss,0.58653


In [ ]:
import wandb

api = wandb.Api()

sweep = api.sweep(f"javierprior04-universidad-de-murcia/tfg-deberta-tuning/{sweep_id}")

best_run = sweep.best_run()

print("Mejor configuración encontrada:")
for key, value in best_run.config.items():
    print(f"  {key}: {value}")

wandb: Sorting runs by -summary_metrics.eval_f1


Mejor configuración encontrada:
  bf16: False
  fp16: False
  fsdp: []
  seed: 42
  tf32: None
  debug: []
  dtype: float16
  optim: adamw_torch_fused
  legacy: True
  do_eval: True
  project: huggingface
  use_cpu: False
  do_train: False
  id2label: {'0': 'SYNTHETIC', '1': 'REAL'}
  label2id: {'REAL': 1, 'SYNTHETIC': 0}
  run_name: None
  data_seed: None
  deepspeed: None
  hub_token: <HUB_TOKEN>
  log_level: passive
  max_steps: -1
  report_to: ['wandb']
  use_cache: False
  adam_beta1: 0.9
  adam_beta2: 0.999
  do_predict: False
  eval_delay: 0
  eval_steps: None
  hidden_act: gelu
  local_rank: -1
  model_type: deberta-v2
  optim_args: None
  output_dir: ./temp_checkpoints
  save_steps: 500
  vocab_size: 128100
  ddp_backend: None
  ddp_timeout: 1800
  fsdp_config: {'xla': False, 'xla_fsdp_v2': False, 'min_num_params': 0, 'xla_fsdp_grad_ckpt': False}
  hidden_size: 768
  label_names: None
  logging_dir: None
  push_to_hub: False
  return_dict: True
  adam_epsilon: 1e-08
  bos_toke

## Entrenar con los mejores hyperparametros

In [ ]:
# --- CELDA DE ENTRENAMIENTO FINAL ---

# 1. Configura aquí los mejores parámetros encontrados por el Sweep
best_config = {
    "learning_rate": 1.4621801161577178e-06,
    "num_train_epochs": 5,
    "batch_size": 8
}

id2label = {0: "SYNTHETIC", 1: "REAL"}
label2id = {"SYNTHETIC": 0, "REAL": 1}

model_final = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

final_args = TrainingArguments(
    seed=42,
    data_seed=42,
    full_determinism=True,
    output_dir="./resultados_finales",
    learning_rate=best_config["learning_rate"],
    num_train_epochs=best_config["num_train_epochs"],
    per_device_train_batch_size=best_config["batch_size"],
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=False
)

trainer = Trainer(
    model=model_final,
    args=final_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

eval_test = trainer.evaluate(tokenized_dataset["test"])
print(f"Resultados en el conjunto de TEST: {eval_test}")

import os

save_path = "/content/drive/MyDrive/TFG/modelos/deberta_v1_sin_limpieza"
# save_path = "/content/drive/MyDrive/TFG/modelos/deberta_v1_con_limpieza"

trainer.save_model(save_path)

Loading weights:   0%|          | 0/198 [00:00<?, ?it/s]

DebertaV2ForSequenceClassification LOAD REPORT from: microsoft/deberta-v3-base
Key                                     | Status     | 
----------------------------------------+------------+-
mask_predictions.classifier.weight      | UNEXPECTED | 
lm_predictions.lm_head.dense.bias       | UNEXPECTED | 
mask_predictions.LayerNorm.bias         | UNEXPECTED | 
lm_predictions.lm_head.bias             | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED | 
lm_predictions.lm_head.dense.weight     | UNEXPECTED | 
mask_predictions.dense.weight           | UNEXPECTED | 
mask_predictions.LayerNorm.weight       | UNEXPECTED | 
mask_predictions.classifier.bias        | UNEXPECTED | 
mask_predictions.dense.bias             | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED | 
classifier.bias                         | MISSING    | 
classifier.weight                       | MISSING    | 
pooler.dense.bias                       | MISSING    | 
pooler.dense.weight      

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.642139,0.666667,0.666667
2,No log,0.624608,0.666667,0.666667
3,No log,0.605926,0.833333,0.857143
4,No log,0.589577,0.833333,0.857143
5,No log,0.583717,0.833333,0.857143


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['deberta.embeddings.LayerNorm.weight', 'deberta.embeddings.LayerNorm.bias', 'deberta.encoder.layer.0.attention.output.LayerNorm.weight', 'deberta.encoder.layer.0.attention.output.LayerNorm.bias', 'deberta.encoder.layer.0.output.LayerNorm.weight', 'deberta.encoder.layer.0.output.LayerNorm.bias', 'deberta.encoder.layer.1.attention.output.LayerNorm.weight', 'deberta.encoder.layer.1.attention.output.LayerNorm.bias', 'deberta.encoder.layer.1.output.LayerNorm.weight', 'deberta.encoder.layer.1.output.LayerNorm.bias', 'deberta.encoder.layer.2.attention.output.LayerNorm.weight', 'deberta.encoder.layer.2.attention.output.LayerNorm.bias', 'deberta.encoder.layer.2.output.LayerNorm.weight', 'deberta.encoder.layer.2.output.LayerNorm.bias', 'deberta.encoder.layer.3.attention.output.LayerNorm.weight', 'deberta.encoder.layer.3.attention.output.LayerNorm.bias', 'deberta.encoder.layer.3.output.LayerNorm.weight', 'deberta.encoder.layer.3.output.Laye

Resultados en el conjunto de TEST: {'eval_loss': 0.4825645089149475, 'eval_accuracy': 0.875, 'eval_f1': 0.8571428571428571, 'eval_runtime': 0.2052, 'eval_samples_per_second': 38.98, 'eval_steps_per_second': 4.872, 'epoch': 5.0}


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]